In [1]:
# binary classification, breast cancer dataset, label and one hot encoded
from numpy import column_stack
from pandas import read_csv
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

/home/super/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
filename="/home/super/Desktop/data/datasets-uci-breast-cancer.csv"

In [3]:

# load data
data = read_csv(filename, header=None)
dataset = data.values
# split data into X and y
X = dataset[:,0:9]
Y = dataset[:,9]




In [4]:
print X.shape
print type(X)

(286, 9)
<type 'numpy.ndarray'>


In [5]:
data.head()


,0,1,2,3,4,5,6,7,8,9
0,'40-49','premeno','15-19','0-2','yes','3','right','left_up','no','recurrence-events'
1,'50-59','ge40','15-19','0-2','no','1','right','central','no','no-recurrence-events'
2,'50-59','ge40','35-39','0-2','no','2','left','left_low','no','recurrence-events'
3,'40-49','premeno','35-39','0-2','yes','3','right','left_low','yes','no-recurrence-events'
4,'40-49','premeno','30-34','3-5','yes','2','left','right_up','no','recurrence-events'


In [6]:
#label encoding . convert into numbers
label_encoder = LabelEncoder()
#print X[:,1].shape

feature = label_encoder.fit_transform(X[:,1])
print type(feature)
print feature[1:10]

<type 'numpy.ndarray'>
[0 0 2 2 2 0 2 2 0]


In [7]:
#train label encoder
labelfit=label_encoder.fit(X[:,1])
#see what it learned
print labelfit.classes_
print "inverse transform [0,1,2] :",labelfit.inverse_transform([0,1,2])

#apply transformation to convert into integer
feature=labelfit.transform(X[:,1])
print feature[0:3]
print feature.shape

#convert row into column shape
feature = feature.reshape(X.shape[0], 1)

print feature[1:3]
print feature.shape

#convert each integer into one hot vector representation 
onehot_encoder = OneHotEncoder(sparse=False)
feature = onehot_encoder.fit_transform(feature)
print feature[1:5]



["'ge40'" "'lt40'" "'premeno'"]
inverse transform [0,1,2] : ["'ge40'" "'lt40'" "'premeno'"]
[2 0 0]
(286,)
[[0]
 [0]]
(286, 1)
[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]]


In [8]:
# encode string input values as integers for all columns
columns = []
for i in range(0, X.shape[1]):
    label_encoder = LabelEncoder()
    feature = label_encoder.fit_transform(X[:,i])
    feature = feature.reshape(X.shape[0], 1)
    
    
    onehot_encoder = OneHotEncoder(sparse=False)
    feature = onehot_encoder.fit_transform(feature)
    columns.append(feature)

/home/super/anaconda2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:216: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [9]:
# collapse columns into array
encoded_x = column_stack(columns)
print type(encoded_x)
print encoded_x

<type 'numpy.ndarray'>
[[ 0.  0.  1. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 ..., 
 [ 0.  1.  0. ...,  1.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  1.  1.  0.]]


In [10]:
#convert output label into numbers
print("X shape: : ", encoded_x.shape)
# encode string class values as integers
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(Y)
label_encoded_y = label_encoder.transform(Y)

('X shape: : ', (286, 43))


In [11]:



# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(encoded_x, label_encoded_y,
test_size=test_size, random_state=seed)
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
print(model)


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)


In [12]:
# make predictions for test data
y_pred = model.predict(X_test)
print y_pred

[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0
 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]


In [13]:
#calculate accuracy

predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 71.58%


In [16]:
from matplotlib import pyplot
from xgboost import plot_tree
import xgboost as xgb


#plot_tree(model)
#pyplot.show()

xgb.to_graphviz(model, num_trees=2)

ImportError: You must install graphviz to plot tree